В папке с программой должен лежать файл quora_question_pairs_rus.csv

In [121]:
from math import log
import numpy as np
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 
import re
import json
from operator import itemgetter
import timeit
import time

In [9]:
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text) 
    text = [pmm.normal_forms(x)[0] for x in text.split()] 
    return text

In [10]:
import csv
with open('quora_question_pairs_rus.csv', 'r', encoding='utf-8') as q:
    str_corpus = csv.reader(q)
    file = list(str_corpus)

In [11]:
file = file[1:]

In [12]:
queries = []
for i in file:
    queries.append(i[1])

In [149]:
d_docs = {}
for i in file:
    docs[i[2]] = i[3]

In [150]:
docs = []
for i in file:
    docs.append([i[2]])

In [19]:
# корпус уже собран и хранится в json файле
#corpus = []
#for i in docs:
#    j = ' '.join(clean_text(i))
#    corpus.append(j)

In [20]:
#with open("doccorpus.json", "w", encoding = 'utf-8') as dc: ## чтоб не собирать корпус каждый раз заново
#    json.dump(corpus, dc)

In [21]:
with open("doccorpus.json", "r", encoding = 'utf-8') as dc:
    copy_corpus = json.load(dc)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [23]:
corpusik = copy_corpus[:10000] ## корпус слишком большой

In [24]:
X = vectorizer.fit_transform(corpusik)
f_matrix = X.toarray()

In [29]:
## Создание матрицы tf-ов
doc_matrix = np.transpose(f_matrix)
doc_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [109]:
## Задание 1. Первый вариант bm25
def bm25(all_n, docs_len, doc_matrix, q_list, corpus, doc, doc_words, k, b, avgdl, N) -> float:
    output = 0
    for word in q_list:
        if word in doc_words:
            w_idx = doc_words.index(word)
            d_idx = corpus.index(doc)
            TF = doc_matrix[w_idx][d_idx]
            n = all_n[word]
            IDF = log((N-n+0.5)/(n+0.5))          
            l_d = docs_len[doc]
            output += IDF * (((TF * (k+1))/(TF + k * (1 - b + (b * (l_d/avgdl))))))
    return output

In [151]:
## Задание 1. Первый вариант bm25 (функция поиска)
def long_search(query):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 0.75
    N = len(corpusik)

    all_n = {}
    for word in doc_words:
        w_idx = doc_words.index(word)
        all_n[word] = np.count_nonzero(doc_matrix[w_idx])
    
    docs_len = {}
    whole_len = 0
    for doc in corpusik:
        doc_len = len(doc.split())
        docs_len[doc] = doc_len
        whole_len += doc_len
    
    avgdl = whole_len/N
    
    q_list = [pmm.normal_forms(x)[0] for x in query.split()] 
    docs_bm = []
    for doc, first_doc in zip(corpusik, docs):
        bm = bm25(all_n, docs_len, doc_matrix, q_list, corpusik, doc, doc_words, k, b, avgdl, N)
        docs_bm.append((first_doc, bm))
    response = sorted(docs_bm,key=itemgetter(1), reverse = True)[:10]
    print(response)
    return response

In [152]:
## Задание 1. Первый вариант bm25 (смотрим время работы на 1 запросе - даже на одном как-то очень долго)
start_time = time.time()
long_search('где я могу найти американскую базу данных семейного офиса')
print("--- %s seconds ---" % (time.time() - start_time))

[(['где я могу найти американскую базу данных семейного офиса'], 43.22957968026908), (['пить более 6 литров воды в день, хорошо или плохо для здоровья'], 37.63025569302986), (['пунктуация: правильно ли добавить вопросительный знак после риторического вопроса'], 13.740498767331104), (['что вы думаете о запрете на 500 и 1000 наименований в Индии?'], 13.543826172798589), (['есть ли способ купить телефон на emi через флипкарт с только дебетовой картой, как'], 12.740266886905324), (['какова была ваша самая большая проблема в качестве государственного секретаря'], 12.330609081755444), (['как преобразовать необработанные файлы в jpeg'], 12.330609081755444), (['сколько способов мы можем создать объект в java'], 12.330609081755444), (['как можно зарабатывать деньги в tutoruniverse com'], 12.330609081755444), (['в чем разница между китайской и западной культурой'], 11.57867848513406)]
--- 68.12321090698242 seconds ---


In [153]:
## Задание 1. Второй вариант bm25
def matr_bm25(all_n, docs_len, doc_matrix, corpus, doc_words, k, b, avgdl, N):
    bm_matrix = []
    for idx, word in enumerate(doc_words):
        bm = 0
        bm_matrix.append([])
        for doc in corpus:
            if word in doc:
                w_idx = doc_words.index(word)
                d_idx = corpus.index(doc)
                TF = doc_matrix[w_idx][d_idx]
                n = all_n[word]
                IDF = log((N-n+0.5)/(n+0.5))          
                l_d = docs_len[doc]
                bm = IDF * ((TF * (k+1))/(TF + k * (1 - b + (b * (l_d/avgdl)))))
            bm_matrix[idx].append(bm)
    return bm_matrix      

In [167]:
## Задание 1. Второй вариант bm25 (функция создания вектора из запроса)
def vect_bm25(query, k, b, doc_words):
    vect = []
    IDF = 1.0986122886681096 ## посчитала отдельно
    q_words = [pmm.normal_forms(x)[0] for x in query.split()]
    for word in doc_words:
        bm = 0
        if word in q_words:
            l_d = len(q_words)
            TF = 1/l_d
            bm = IDF * ((TF * (k+1))/(TF + k))
        vect.append(bm)
    vect = np.array(vect)
    return vect

In [124]:
## получаем матрицу bm'ов для корпуса
bm_matrix = matr_bm25(all_n, docs_len, doc_matrix, corpusik, doc_words, k, b, avgdl, N)
bm_matrix = np.array(bm_matrix)

In [71]:
bm_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 6.78342452, 6.78342452,
        6.78342452],
       [0.        , 0.        , 0.        , ..., 9.12123438, 9.12123438,
        9.12123438],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [172]:
## Задание 1. Второй вариант bm25 (функция поиска)
def search(query, bm_matrix):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 0.75
    q_vect = vect_bm25(query, k, b, doc_words)
    doc_score = q_vect.dot(bm_matrix)
    response = list(zip(docs, doc_score))
    response = sorted(response,key=itemgetter(1), reverse = True)[:10]
    print(response)
    return response

In [173]:
## Задание 1. Второй вариант bm25 (смотрим время работы на 1 запросе) 
## Гораздо быстрее, чем первый, но зато совсем неправильно...... как же так
start_time = time.time()
search('где я могу найти американскую базу данных семейного офиса', bm_matrix)
print("--- %s seconds ---" % (time.time() - start_time))

[(['как я могу эффективно учиться во время сна'], 7.944774760250626), (['вы можете играть в азартные игры 4 на европейской игровой станции 4'], 7.944774760250626), (['кто твой лучший друг, почему'], 7.92808045013364), (['как он себя чувствует'], 7.92808045013364), (['который является Шри Шри Рави Шанкаром'], 7.92808045013364), (['означает ли это что угодно, если девушка посылает вам много сердец в ее текстах'], 7.92808045013364), (['как отношения Индии с американским'], 7.90450828151765), (['Как я могу перестать смотреть порно'], 7.90450828151765), (['почему uk имели плохие жизненные стандарты в 70-х годах 80-х годов, но теперь лучше'], 7.90450828151765), (['как может студенческая помощь в исследовании машинного обучения'], 7.90450828151765)]
--- 0.16111373901367188 seconds ---


In [174]:
## ЗАДАНИЕ 2.
search('рождественские каникулы', bm_matrix)

[(['в охотнике x охотник, который, должно быть, послал орла, чтобы напасть на комуги'], 7.6698738084581), (['что является лучшим вариантом для инвестиций'], 7.6698738084581), (['почему в голландской империи было так мало колоний в XX веке'], 7.6698738084581), (['является хамасом террористической группы'], 7.6698738084581), (['я должен уделить время от знакомства, чтобы сосредоточиться на изменении карьеры. Я 33-летний мужчина'], 7.6698738084581), (['что должен делать парень, когда в середине разговора его девушка переключается на просмотр фильмов, чем разговаривает с ним'], 7.6698738084581), (['почему мой ноутбук перегревается'], 7.6698738084581), (['я страдаю от пограничного расстройства личности, и мой парень не понимает его, как я могу помочь ему понять, поэтому он будет более терпеливым со мной'], 7.6698738084581), (['как вы справляетесь с работой, когда вы и ваша девушка работаете в одном офисе'], 7.6698738084581), (['рабочие, ненавистные или почитаемые в американском обществе'], 

[(['в охотнике x охотник, который, должно быть, послал орла, чтобы напасть на комуги'],
  7.6698738084581),
 (['что является лучшим вариантом для инвестиций'], 7.6698738084581),
 (['почему в голландской империи было так мало колоний в XX веке'],
  7.6698738084581),
 (['является хамасом террористической группы'], 7.6698738084581),
 (['я должен уделить время от знакомства, чтобы сосредоточиться на изменении карьеры. Я 33-летний мужчина'],
  7.6698738084581),
 (['что должен делать парень, когда в середине разговора его девушка переключается на просмотр фильмов, чем разговаривает с ним'],
  7.6698738084581),
 (['почему мой ноутбук перегревается'], 7.6698738084581),
 (['я страдаю от пограничного расстройства личности, и мой парень не понимает его, как я могу помочь ему понять, поэтому он будет более терпеливым со мной'],
  7.6698738084581),
 (['как вы справляетесь с работой, когда вы и ваша девушка работаете в одном офисе'],
  7.6698738084581),
 (['рабочие, ненавистные или почитаемые в амер

Ниже Задание 3. (но я не поняла, как посчитать точность)

In [182]:
def search_bm15(query, bm_matrix):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 0
    q_vect = vect_bm25(query, k, b, doc_words)
    doc_score = q_vect.dot(bm_matrix)
    response = list(zip(docs, doc_score))
    response = sorted(response,key=itemgetter(1), reverse = True)[:10]
    return response

In [183]:
def search_bm11(query, bm_matrix):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 1
    q_vect = vect_bm25(query, k, b, doc_words)
    doc_score = q_vect.dot(bm_matrix)
    response = list(zip(docs, doc_score))
    response = sorted(response,key=itemgetter(1), reverse = True)[:10]
    return response

In [184]:
response = search('где я могу найти американскую базу данных семейного офиса', bm_matrix)
response

[(['как я могу эффективно учиться во время сна'], 7.944774760250626), (['вы можете играть в азартные игры 4 на европейской игровой станции 4'], 7.944774760250626), (['кто твой лучший друг, почему'], 7.92808045013364), (['как он себя чувствует'], 7.92808045013364), (['который является Шри Шри Рави Шанкаром'], 7.92808045013364), (['означает ли это что угодно, если девушка посылает вам много сердец в ее текстах'], 7.92808045013364), (['как отношения Индии с американским'], 7.90450828151765), (['Как я могу перестать смотреть порно'], 7.90450828151765), (['почему uk имели плохие жизненные стандарты в 70-х годах 80-х годов, но теперь лучше'], 7.90450828151765), (['как может студенческая помощь в исследовании машинного обучения'], 7.90450828151765)]


[(['как я могу эффективно учиться во время сна'], 7.944774760250626),
 (['вы можете играть в азартные игры 4 на европейской игровой станции 4'],
  7.944774760250626),
 (['кто твой лучший друг, почему'], 7.92808045013364),
 (['как он себя чувствует'], 7.92808045013364),
 (['который является Шри Шри Рави Шанкаром'], 7.92808045013364),
 (['означает ли это что угодно, если девушка посылает вам много сердец в ее текстах'],
  7.92808045013364),
 (['как отношения Индии с американским'], 7.90450828151765),
 (['Как я могу перестать смотреть порно'], 7.90450828151765),
 (['почему uk имели плохие жизненные стандарты в 70-х годах 80-х годов, но теперь лучше'],
  7.90450828151765),
 (['как может студенческая помощь в исследовании машинного обучения'],
  7.90450828151765)]

In [185]:
response = search_bm15('где я могу найти американскую базу данных семейного офиса', bm_matrix)
response

[(['как я могу эффективно учиться во время сна'], 7.944774760250626),
 (['вы можете играть в азартные игры 4 на европейской игровой станции 4'],
  7.944774760250626),
 (['кто твой лучший друг, почему'], 7.92808045013364),
 (['как он себя чувствует'], 7.92808045013364),
 (['который является Шри Шри Рави Шанкаром'], 7.92808045013364),
 (['означает ли это что угодно, если девушка посылает вам много сердец в ее текстах'],
  7.92808045013364),
 (['как отношения Индии с американским'], 7.90450828151765),
 (['Как я могу перестать смотреть порно'], 7.90450828151765),
 (['почему uk имели плохие жизненные стандарты в 70-х годах 80-х годов, но теперь лучше'],
  7.90450828151765),
 (['как может студенческая помощь в исследовании машинного обучения'],
  7.90450828151765)]

In [186]:
response = search_bm11('где я могу найти американскую базу данных семейного офиса', bm_matrix)
response

[(['как я могу эффективно учиться во время сна'], 7.944774760250626),
 (['вы можете играть в азартные игры 4 на европейской игровой станции 4'],
  7.944774760250626),
 (['кто твой лучший друг, почему'], 7.92808045013364),
 (['как он себя чувствует'], 7.92808045013364),
 (['который является Шри Шри Рави Шанкаром'], 7.92808045013364),
 (['означает ли это что угодно, если девушка посылает вам много сердец в ее текстах'],
  7.92808045013364),
 (['как отношения Индии с американским'], 7.90450828151765),
 (['Как я могу перестать смотреть порно'], 7.90450828151765),
 (['почему uk имели плохие жизненные стандарты в 70-х годах 80-х годов, но теперь лучше'],
  7.90450828151765),
 (['как может студенческая помощь в исследовании машинного обучения'],
  7.90450828151765)]